#  Stacking for the estimation of Treatment Effects

# 1. Generate data
generate data according to the same process as Nie X. and Wager S. (2018) 'Quasi-Oracle Estimation of Heterogeneous Treatment Effects'. a library is provided by `causalml` from uber (https://github.com/uber/causalml). 

the goal in simulating data : to provide different examples of data generating processes, in order to conclude upon the effectiveness of stacking for treatment effects in each situation. 

In an experimental setup, or a situation in which we have treated and untreated data, it is necessary to estimate the underlying distribution of the 'nuisance variables', the propensity score (the likelihood, given an observation's characteristics, to be treated), and the underlying treatment effect. As such, we simulate different X, propensity, and treatment functions.

`causalml` provides an implementation of each data generating function as seen in Nie & Wager, accessible through five possible modes passed to synthetic_data() :          

    `       1 for difficult nuisance components and an easy treatment effect.
            2 for a randomized trial.
            3 for an easy propensity and a difficult baseline.
            4 for unrelated treatment and control groups.
            5 for a hidden confounder biasing treatment.

!!!! talk here about how its useful to use the data generators from causal ml bc :
 - this allows us to generate as much data as we need in order to cross validate. we essentially always have a holdout set read to test.
 - we also need true treatment effects in order to set up a stacking pipeline, there is otherwise no way to estimate a stacking model without a true $\tau(X)$

In [1]:
### YOU COULD DO THE DATA GENERATION THING HERE WITH CAUSALML, AND THEN CREATE A PIPELINE ENTIRELY WITH SKLEARN.
from causalml.dataset import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

np.seterr(divide='ignore', invalid='ignore')
np.random.seed(42)

### THESE NEED A RANDOM SEED IF YOU WANT TO USE THEM ON THE R LEARNER AND COMPARE.
### nb: b is the expected outcome, (i guess without treatment? is like a counterfactual??? unsure.)


y_a, X_a, w_a, tau_a, b_a, e_a = simulate_nuisance_and_easy_treatment()
#y_b, X_b, w_b, tau_b, b_b, e_b = simulate_randomized_trial()
#y_c, X_c, w_c, tau_c, b_c, e_c = simulate_easy_propensity_difficult_baseline()
#y_e, X_e, w_e, tau_e, b_e, e_e = simulate_hidden_confounder()

### the simple learners with just sklearn and some ensemble methods.
### from this blog post : https://florianwilhelm.info/2017/04/causal_inference_propensity_score/

const = np.ones(1000) ## default for causalml synthetic data gen funcs.
df = pd.DataFrame(data=X_a)
df['assignment'] = w_a
df['outcome'] = y_a
df.head()

reg = RandomForestRegressor()
X = df.iloc[:, 0:5]
y = df['outcome']

reg.fit(X, y)

X_neg = pd.DataFrame(data=X)
X_neg['assignment'] = 0

X_pos = pd.DataFrame(data=X)
X_pos['assignment'] = 1

#preds_rnd = reg.predict(X_pos) / reg.predict(X_neg)

thing = reg.predict(X_pos)
###reg.predict(X_pos)

The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.


ValueError: Number of features of the model must match the input. Model n_features is 5 and input n_features is 6 

# 2. Learner selection -

## Grimmer et al, the 'T learner' and OLS.
Nie and Wager estimate treatement effects using (among other heterogenous treatment effect estimators) the 'T-Learner', we take this learner to be the same as that which is used in Grimmer (2017)
`causalml library also provides a T-learner as used in Grimmer (2017), where the treatment effect is estimated by simple comparison of the treated against the untreated.

The T Learner fits the functions $ \mu^*_w(x) = E(Y\mid X = x, W = w)$ separately for the treatment and the control groups (for $w \in{0, 1}$), and then esimates $\tau(x) = \mu_1(x) - \mu_0(x)$
The T learner (`BaseTRegressor()`) as implemented by `causalml` can be fed a machine learning model to estimate and predict treatment effects.
it will therefore be useful to have a stacking pipeline which includes several T-Learners, over different datasets in order to examine how well a Stacked T learner does under different data generating functions.

we can use this to stack estimates exactly as in Grimmer, and compare them to the R learners/DML learnings (also stacked) as in Nie, Wager
this would answer the questions:
 - how does stacking **without** double machine learning compare to the R learner on its own? Is there an advantage to using (the best..) R learner over simple stacking? and what kind of libraries are already available for such a pipeline?
 - how much can stacking help if we do not (can not) estimate propensity scores or distinct nuisance funcitions in the dataset?
 - how does stacking an R learner compare to (the best) single DML model?

the T learner does not explicitly account for propensity scores, or for the 'nuisance funciton' separate from the treatment effect.

the R learner, on the other hand:
 - R leaner, DML, etc etc.
 - The proposal from Nie and Wager for using some 'out of the box models and stacking the treatment effect estimations

The R learner can also be fed different models, and the results similarly stacked.

# 2. Feed in different models

We continue in the methodology of Nie and Wager, (QUOTE), by creating R learners using various methods to estimate the underlying functions for the X variables. 
(for the moment we will always use ElasticNetPropensityModel to estimate propensity scores.)

How many models can we feed in?

In [ ]:
from causalml.inference.meta import BaseRRegressor, BaseTRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import statsmodels.api as sm
import copy
import numpy as np

In [ ]:
## can start with estimates from the T learner

learner_t_xgb = BaseTRegressor(learner=XGBRegressor())
learner_t_mlpr = BaseTRegressor(learner=MLPRegressor())
learner_t_lr = BaseTRegressor(learner=LinearRegression())

estimators = {'learner_t_xgb' : BaseTRegressor(learner=XGBRegressor()),
              'learner_t_mlpr': BaseTRegressor(learner=MLPRegressor()),
              'learner_t_lr' : BaseTRegressor(learner=LinearRegression())}

predicitons_t_models = get_synthetic_preds(simulate_nuisance_and_easy_treatment,
                                               n=50000,
                                               estimators=estimators)

In [ ]:
'''
estimators = {'learner_xgb': BaseRRegressor(learner=XGBRegressor()),
              'learner_lr': BaseRRegressor(learner=LinearRegression()),
              'learner_dtr': BaseRRegressor(learner=DecisionTreeRegressor()),
              'learner_ctr': BaseRRegressor(learner=CausalTreeRegressor()),
              'learner_knr': BaseRRegressor(learner=KNeighborsRegressor()),
              'learner_svr': BaseRRegressor(learner=SVR())}
              '''
###would be cool to find some other working learners, and to start messing with the params of each!!!!
#learner_knr = BaseRRegressor(learner=KNeighborsRegressor())
#learner_svr = BaseRRegressor(learner=SVR())
#learner_ctr = BaseRRegressor(learner=CausalTreeRegressor())
#'learner_sgd': BaseRRegressor(learner=SGDRegressor())
#learner_nnr = BaseRRegressor(learner=MLPRegressor()) ##Multi-layer Perceptron regressor

estimators = {'learner_xgb': BaseRRegressor(learner=XGBRegressor()),
              'learner_lr': BaseRRegressor(learner=LinearRegression()),
              'learner_dtr': BaseRRegressor(learner=DecisionTreeRegressor())}

predictions = get_synthetic_preds(simulate_nuisance_and_easy_treatment,
                                               n=50000,
                                               estimators=estimators)

In [ ]:
### THE STADCKING PART (AGAIN.)

y_stacking = predictions['generated_data']['tau']
pred_copy = copy.deepcopy(predictions)
pred_copy.pop('Actuals')
pred_copy.pop('generated_data')

### iterate over this better so you can add mroe models.
x_stacking = np.vstack((predictions['learner_xgb'], predictions['learner_lr'], predictions['learner_dtr'])).T

### this is using ridgeCV for the method of ols.
### grimmer require that the coefficients of the final stacking regression sum to 1.
### HOW CAN WE DO THIS IN SKLEARN WHILE STILL USING CAUSALML?

model = sm.OLS(y_stacking, x_stacking)
model2 = model.fit_regularized(alpha=0.0, L1_wt=1.0, start_params=None, profile_scale=False, refit=False)
print(model2.params)
print(sum(model2.params))

this is actually a sad shortcoming of the `causalml` library, that it is not possible to feed models directly into sklearn.ensemble models.
normally an ensemble class from sklear woudl take a dict (i think..) of models and a dataset, do the entire pipeline, and then output the model.

if we would like to imitate grimmer we would also need a solution to the problem that we currently dont have a way to contstrain the ensemble coefficients such that they must sum to 1.

In [ ]:
predictions_A = get_synthetic_preds(simulate_nuisance_and_easy_treatment, n=50000)
predictions_B = get_synthetic_preds(simulate_randomized_trial, n=50000)
predictions_C = get_synthetic_preds(simulate_easy_propensity_difficult_baseline, n=50000)
predictions_E = get_synthetic_preds(simulate_hidden_confounder, n=50000)